# Scrap personnalités politiques Fr

In [1]:
import bs4
import urllib
import pandas as pd
import re

from urllib import request

On utilise une page wikipédia qui liste les ministres de la Ve république

In [7]:
url_wiki_ministres_1 = "https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:Ministre_de_la_Cinqui%C3%A8me_R%C3%A9publique"
url_wiki_ministres_2 = "https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Ministre_de_la_Cinqui%C3%A8me_R%C3%A9publique&pageuntil=Paul%2C+Christian%0AChristian+Paul+%28homme+politique%29#mw-pages"
url_wiki_ministres_3 = "https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Ministre_de_la_Cinqui%C3%A8me_R%C3%A9publique&pagefrom=Paul%2C+Christian%0AChristian+Paul+%28homme+politique%29#mw-pages"

URL=[url_wiki_ministres_1,url_wiki_ministres_2,url_wiki_ministres_3]

Ministres=[]
for url in URL:
    request_text = request.urlopen(url).read()
    page = bs4.BeautifulSoup(request_text, "lxml")

    mydivs = page.find_all("div", {"class": "mw-category"})
    Ministres+=[element.get('title') for element in mydivs[0].find_all('a')]

# On retire les '(homme/femme politique,..)' de certains noms
Ministres_Ve=[re.sub('[\(].*?[\)]','',ministre) for ministre in Ministres]

On utilise la page Wikipédia qui liste les présidents de la République (on se restreint à la Ve République).

In [13]:
url_wiki_president="https://fr.wikipedia.org/wiki/Liste_des_pr%C3%A9sidents_de_la_R%C3%A9publique_fran%C3%A7aise"
request_text = request.urlopen(url_wiki_president).read()
page=bs4.BeautifulSoup(request_text, "lxml")

mydivs=page.find_all('table', {"class": "wikitable"})[-1]
Presidents=[element.get('title') for element in mydivs.find_all('a')[1:]]

In [14]:
Presidents

['Charles de Gaulle',
 'Jacques Chirac',
 'Georges Pompidou',
 'François Mitterrand',
 "Valéry Giscard d'Estaing",
 'Nicolas Sarkozy',
 'Emmanuel Macron',
 'François Hollande']

On récupère aussi l'ensemble des parlementaires via https://data.assemblee-nationale.fr/acteurs/deputes-en-exercice et
https://data.senat.fr/les-senateurs/

In [26]:
df = pd.read_csv('liste_deputes_libre_office.csv')

Deputes = [df.loc[i]['Prénom'] + ' ' + df.loc[i]['Nom'] for i in range(len(df)) ]

In [2]:
df = pd.read_excel('liste_senateurs.xls')
df_actif = df[df['État']=='ACTIF'].reset_index()

Senateurs = [df_actif.loc[i]['Prénom usuel'] + ' ' + df_actif.loc[i]['Nom usuel'] for i in range(len(df_actif)) ]

On rajoute les députés européens élus en 2019

In [6]:
url_wiki_eurodeputes="https://fr.wikipedia.org/wiki/%C3%89lections_europ%C3%A9ennes_de_2019_en_France"
request_text = request.urlopen(url_wiki_eurodeputes).read()
page=bs4.BeautifulSoup(request_text, "lxml")

eurodeputes=[]
for i in range(1,7):
    length = len(page.find_all('div', {'class': 'colonnes'})[-i].find_all('li'))
    eurodeputes += [page.find_all('div', {'class': 'colonnes'})[-i].find_all('li')[j].find('a').get('title') for j in range(length)]

In [7]:
len(eurodeputes)

79

Si l'on veut les candidats aux élections présidentielles 2017:

In [146]:
url_candidats_2017 = "https://fr.wikipedia.org/wiki/%C3%89lection_pr%C3%A9sidentielle_fran%C3%A7aise_de_2017#Candidats"
request_text = request.urlopen(url_candidats_2017).read()
page = bs4.BeautifulSoup(request_text, "lxml")

mydivs=page.find_all('table', {"class": "wikitable"})[-1]
Candidats_2017=[element.get('title') for element in mydivs.find_all('a')]

while None in Candidats_2017:
    Candidats_2017.remove(None)